In [1]:
# Random Imports
import os
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

## Initialize the runner used to run jobs.

In [2]:
# Submit cell
from ese.experiment.experiment.ese_exp import CalibrationExperiment 
from ionpy.slite import SliteRunner

# List the available gpus for a machine
available_gpus = ['0', '1', '2', '3']

# Configure Slite Object
srunner = SliteRunner(
    project='ESE',
    exp_name='debug',
    task_type=CalibrationExperiment, 
    available_gpus=available_gpus
    )

Initalized SliteRunner


## Define said jobs

In [3]:
from ionpy.util.config import check_missing

def validate_cfg(cfg):
    # It's usually a good idea to do a sanity check of
    # inter-related settings or force them manually
    check_missing(cfg)        
    return cfg

In [4]:
%%yaml default_cfg 

experiment:
  seed: 42
    
dataloader:
  batch_size: 1 
  num_workers: 4 
  pin_memory: False 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 
  
train:
  epochs: 500 
  eval_freq: 50
  
log:
  checkpoint_freq: 50 
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 

loss_func: 
  _class: ionpy.loss.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [5]:
%%yaml model_cfg  

model:
  _class: ese.experiment.models.UNet
  in_channels: 1
  out_channels: 1
  filters: '?'
  convs_per_block: 3

<IPython.core.display.Javascript object>

In [6]:
%%yaml dataset_cfg 

# data:
#   _class: ese.experiment.datasets.WMH
#   annotator: observer_o12
#   axis: 0
#   preload: False 
#   num_slices: 1
#   task: Amsterdam 
#   version: 0.2

data:
  _class: ese.experiment.datasets.COCO 

<IPython.core.display.Javascript object>

In [7]:
%%yaml lite_aug_cfg

- RandomAffine:
    p: 0.5
    degrees: [0, 360]
    translate: [0, 0.2]
    scale: [0.8, 1.1]
- RandomVariableElasticTransform:
    p: 0.75
    alpha: [1, 2] 
    sigma: [7, 9]
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5

<IPython.core.display.Javascript object>

In [8]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [9]:
from ionpy.util import Config
import pickle
import copy

# Assemble base config
base_cfg = Config(default_cfg).update(model_cfg).update(dataset_cfg).update(callbacks_cfg)

# Save the base cfg and the lite aug cfg
with open('/storage/vbutoi/projects/ESE/configs/base.pkl', 'wb') as base_config_file:
    pickle.dump(base_cfg, base_config_file)
    
with open('/storage/vbutoi/projects/ESE/configs/lite_aug.pkl', 'wb') as aug_config_file:
    pickle.dump(sum([copy.deepcopy(lite_aug_cfg)], start=[]), aug_config_file)

## Debug Station

In [10]:
# Need to define the experiment name
exp_name = 'debug'

# Create the ablation options
option_set = [
    {
        'log.root': ['/storage/vbutoi/scratch/ESE/debug'],
        'dataloader.batch_size': [1],
        'model.filters': [[128, 128, 128, 128, 128]],
    },
]

In [11]:
from ese.scripts.utils import get_option_product
import copy

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

# Add Augmentations
for cfg_idx in range(len(cfgs)):
    for add_aug in [True, False]:
        if add_aug:
            cfgs[cfg_idx] = cfgs[cfg_idx].set('augmentations', sum([copy.deepcopy(lite_aug_cfg)], start=[]))

In [12]:
srunner.run_exp(cfgs[0])

loading annotations into memory...
Done (t=16.23s)
creating index...
index created!
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
Running CalibrationExperiment("/storage/vbutoi/scratch/ESE/debug/20230907_103456-RYZ3-3c18965714bdaeeea23447e4e08d356e")
---
augmentations:
- RandomAffine:
    degrees:
    - 0
    - 360
    p: 0.5
    scale:
    - 0.8
    - 1.1
    translate:
    - 0
    - 0.2
- RandomVariableElasticTransform:
    alpha:
    - 1
    - 2
    p: 0.75
    sigma:
    - 7
    - 9
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5
callbacks:
  epoch:
  - ese.experiment.callbacks.WandbLogger
  - ionpy.callbacks.ETA
  - ionpy.callbacks.JobProgress
  - ionpy.callbacks.TerminateOnNaN
  - ionpy.callbacks.PrintLogged
  - ionpy.callbacks.ModelCheckpoint:
      monitor: dice_score
      phase: val
  step:
  - ese.experiment.callbacks.ShowPredictions
dat

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vbutoi. Use `wandb login --relogin` to force relogin


Start epoch 0


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 143, in <listcomp>
    return [coll